steps to implement:
1. simple qa chat bot implement karna h.
2. Then ek chatbot implement karna h jo user k first message ko around conversation ko name dega.
3. Chat ka name print karwana h aur ans bhi

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
chatmodel = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
namemodel = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)

In [4]:
name_instruction = """
You are an assistant, whose task is the analyse the first message of the user 
and generate a name for the convesation. Just write the name only that you have generated nothing else.  
"""
name_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", name_instruction),
        ("user", "{input}")
    ]
)

In [5]:
instruction = """
You are an intelligent assistant, your task is to answer the user querries as best as you can.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruction),
        ("user","{input}")
    ]
)

In [6]:
parser = StrOutputParser()

In [7]:
chain = prompt|chatmodel|parser
name_chain = name_prompt|namemodel|parser

In [8]:
querry = input("Enter your question")
name = name_chain.invoke({"input":querry})
print("Convestion Name :", name)
response = chain.invoke({"input":querry})
print(response)

Convestion Name : Let's Chat! 


I understand! I'm ready to answer your questions and do my best to be helpful. Ask me anything!  

I can help with things like:

* **Providing information:** I have access to a vast store of knowledge and can answer questions on a variety of topics.
* **Generating creative content:** I can write stories, poems, articles, and even code.
* **Summarizing text:** Need to get the gist of a long article? I can do that for you.
* **Translating languages:** I can translate text between many different languages.

Let me know how I can assist you! 😊  




In [9]:
name.strip()

"Let's Chat!"

In [10]:
response.strip()

"I understand! I'm ready to answer your questions and do my best to be helpful. Ask me anything!  \n\nI can help with things like:\n\n* **Providing information:** I have access to a vast store of knowledge and can answer questions on a variety of topics.\n* **Generating creative content:** I can write stories, poems, articles, and even code.\n* **Summarizing text:** Need to get the gist of a long article? I can do that for you.\n* **Translating languages:** I can translate text between many different languages.\n\nLet me know how I can assist you! 😊"

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

hist_chain = prompt|chatmodel
# Function to get seesion chats
store = {}
def get_session_history(sessionID:str)->BaseChatMessageHistory:
    if sessionID not in store:
        store[sessionID] = ChatMessageHistory()
    return store[sessionID]

message_hist_chain = RunnableWithMessageHistory(
    hist_chain,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

ans = message_hist_chain.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "foo"}}
    )

store

{'foo': InMemoryChatMessageHistory(messages=[HumanMessage(content='What does cosine mean?', additional_kwargs={}, response_metadata={}), AIMessage(content='I\'m happy to explain cosine!\n\n**Cosine (cos)** is a trigonometric function that relates an angle of a right triangle to the ratio of two of its sides. \n\nHere\'s the breakdown:\n\n* **Right Triangle:**  We\'re talking about a triangle with one angle measuring 90 degrees.\n\n* **Adjacent and Hypotenuse:**\n    * **Adjacent** side: The side next to the angle you\'re considering (but not the hypotenuse).\n    * **Hypotenuse:** The longest side of the triangle, opposite the right angle.\n\n* **The Formula:**  \n   cos(angle) = Adjacent side / Hypotenuse\n\n**In simpler words:** Cosine tells you how "stretched out" or "compressed" the adjacent side is compared to the hypotenuse of a right triangle, based on the angle.\n\n**Let me know if you\'d like to see a visual example or have any other math questions!**\n', additional_kwargs={},

In [12]:
ans = message_hist_chain.invoke(
    {"ability": "math", "question": "Why we always take the ratio of base and hypotenuse of the right angled triangle"},
    config={"configurable": {"session_id": "foo"}}
    )
ans.content

'That\'s a great question! It gets to the heart of why we have different trigonometric functions. \n\nWe use the ratio of the base (adjacent side) to the hypotenuse for cosine specifically because it highlights a particular relationship in the triangle. \n\nHere\'s why it\'s a useful ratio:\n\n* **Angle Relationship:** The cosine of an angle directly relates to how much the adjacent side is "stretched" or "compressed" compared to the hypotenuse. As the angle changes, the ratio of the base to the hypotenuse changes accordingly.\n\n* **Solving for Unknown Sides:** If you know the angle and the hypotenuse of a right triangle, you can use cosine to find the length of the adjacent side.\n\n* **Geometric Applications:** Cosine is fundamental in many geometric applications, such as:\n    * **Finding distances:**  In surveying or navigation, cosine can help calculate distances based on angles and known lengths.\n    * **Understanding forces:** In physics, cosine is used to analyze the componen

In [13]:
store["foo"].messages

[HumanMessage(content='What does cosine mean?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I\'m happy to explain cosine!\n\n**Cosine (cos)** is a trigonometric function that relates an angle of a right triangle to the ratio of two of its sides. \n\nHere\'s the breakdown:\n\n* **Right Triangle:**  We\'re talking about a triangle with one angle measuring 90 degrees.\n\n* **Adjacent and Hypotenuse:**\n    * **Adjacent** side: The side next to the angle you\'re considering (but not the hypotenuse).\n    * **Hypotenuse:** The longest side of the triangle, opposite the right angle.\n\n* **The Formula:**  \n   cos(angle) = Adjacent side / Hypotenuse\n\n**In simpler words:** Cosine tells you how "stretched out" or "compressed" the adjacent side is compared to the hypotenuse of a right triangle, based on the angle.\n\n**Let me know if you\'d like to see a visual example or have any other math questions!**\n', additional_kwargs={}, response_metadata={'token_usage': {'comple

In [14]:
## General functions
import json
import os
from typing import List
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage

class JSONChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, file_path: str = "../chat_sessions.json"):
        self.session_id = session_id
        self.file_path = file_path
        self.messages = self._load_messages()

    def _load_messages(self) -> List[BaseMessage]:
        if not os.path.exists(self.file_path):
            return []
        with open(self.file_path, "r") as f:
            all_data = json.load(f)
        session_data = all_data.get(self.session_id, [])
        return [self._dict_to_message(m) for m in session_data]

    def _save_messages(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, "r") as f:
                all_data = json.load(f)
        else:
            all_data = {}

        all_data[self.session_id] = [self._message_to_dict(m) for m in self.messages]

        with open(self.file_path, "w") as f:
            json.dump(all_data, f, indent=2)

    def add_message(self, message: BaseMessage) -> None:
        self.messages.append(message)
        self._save_messages()

    def clear(self) -> None:
        self.messages = []
        self._save_messages()

    def _message_to_dict(self, message: BaseMessage):
        return {"type": message.type, "content": message.content}

    def _dict_to_message(self, data: dict) -> BaseMessage:
        if data["type"] == "human":
            return HumanMessage(content=data["content"])
        elif data["type"] == "ai":
            return AIMessage(content=data["content"])
        else:
            raise ValueError("Unknown message type")


In [15]:
## General functions
import json
import os
from typing import List
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage

class JSONChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, file_path: str = "../chat_sessions.json"):
        self.session_id = session_id
        self.file_path = file_path
        self.messages = self._load_messages()

    def _load_messages(self) -> List[BaseMessage]:
        if not os.path.exists(self.file_path):
            return []
        with open(self.file_path, "r") as f:
            all_data = json.load(f)
        session_data = all_data.get(self.session_id, [])
        return [self._dict_to_message(m) for m in session_data]

    def _save_messages(self):
        if os.path.exists(self.file_path):
            with open(self.file_path, "r") as f:
                all_data = json.load(f)
        else:
            all_data = {}

        all_data[self.session_id] = [self._message_to_dict(m) for m in self.messages]

        with open(self.file_path, "w") as f:
            json.dump(all_data, f, indent=2)

    def add_message(self, message: BaseMessage) -> None:
        self.messages.append(message)
        self._save_messages()

    def clear(self) -> None:
        self.messages = []
        self._save_messages()

    def _message_to_dict(self, message: BaseMessage):
        return {"type": message.type, "content": message.content}

    def _dict_to_message(self, data: dict) -> BaseMessage:
        if data["type"] == "human":
            return HumanMessage(content=data["content"])
        elif data["type"] == "ai":
            return AIMessage(content=data["content"])
        else:
            raise ValueError("Unknown message type")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq

# Set up the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant skilled in {ability}."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# Set up the model
llm = ChatGroq(model="gemma2-9b-it")  # or llama3-8b-8192

# Combine into a chain
chain = prompt | llm

# History setup
def get_session_history(session_id: str):
    return JSONChatMessageHistory(session_id=session_id, file_path="../chat_sessions.json")

# Wrap chain with memory
message_chain = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history"
)

response = message_chain.invoke(
    {"ability": "math", "question": "What is cosine?"},
    config={"configurable": {"session_id": "math_test"}}
)

print(response.content)


You're right to ask again! My previous explanation was a bit technical.  

Imagine a right triangle, like the corner of a square. 

* **Cosine** is a way to measure how "stretched out" or "squished" the triangle is, depending on the angle you choose.

Think of it like this:

* **Angle:** Choose one of the angles in the triangle (except the 90-degree one).
* **Adjacent Side:** Look at the side next to the angle you chose.
* **Hypotenuse:** This is the longest side, always opposite the right angle.

Cosine tells you the ratio of the adjacent side's length to the hypotenuse's length. 

*  A cosine of 1 means the triangle is very "stretched out" – the adjacent side is almost as long as the hypotenuse.
*  A cosine of 0 means the triangle is very "squished" – the adjacent side is very short.


Let me know if you'd like me to draw a picture or give you another example!



In [16]:
response = message_chain.invoke(
    {"ability": "Expert in handling people", "question": "I had applied for a job role a week later draft a mail asking about the hr is there is any update for my application"},
    config={"configurable": {"session_id": "people-handler"}}
)

In [17]:
response

AIMessage(content="Subject: Following Up on Application for [Job Title] - [Your Name]\n\nDear [Hiring Manager name],\n\nI hope this email finds you well.\n\nI'm writing to follow up on my application for the [Job Title] position that I submitted last week on [Date you submitted application]. I'm very interested in this opportunity and believe my skills and experience in [mention 1-2 relevant skills] would be a great fit for your team.\n\nI understand that you're likely reviewing many applications, but I wanted to check in and see if there are any updates on the hiring timeline.\n\nThank you for your time and consideration.\n\nSincerely,\n\n[Your Name] \n\n\n\n**Key Points:**\n\n* **Reference the application date:** This helps the hiring manager quickly identify your application.\n* **Keep it concise:** Hiring managers appreciate brevity.\n* **Express continued interest:** Reiterate your enthusiasm for the role.\n* **Be respectful of their time:** Acknowledge the volume of applications 

In [ ]:
from datetime import datetime
def generate_session_id():
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M:%S")
    return [f"session_{timestamp}", date, time]

In [23]:
details = generate_session_id()
details1 = generate_session_id()
details2 = generate_session_id()
details3 = generate_session_id()
details4 = generate_session_id()

In [24]:
details[0], details1[0], details2[0], details3[0], details4[0]

('session_20250802093733652192',
 'session_20250802093733652192',
 'session_20250802093733652192',
 'session_20250802093733652192',
 'session_20250802093733652192')

In [28]:
import random, string
def generate_session_details():
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    rand_suffix = "".join(random.choices(string.ascii_lowercase + string.digits, k=6))
    session_id = f"session_{timestamp}-{rand_suffix}"
    date = datetime.now().strftime("%Y-%m-%d")
    time = datetime.now().strftime("%H:%M:%S")
    return [session_id, date, time]

In [29]:
details = generate_session_id()
details1 = generate_session_id()
details2 = generate_session_id()
details3 = generate_session_id()
details4 = generate_session_id()

In [30]:
details[0], details1[0], details2[0], details3[0], details4[0]

('session_20250802094353287473-61k2ia',
 'session_20250802094353287473-jms6gr',
 'session_20250802094353287473-3cq1on',
 'session_20250802094353287473-rxg1q3',
 'session_20250802094353288561-0zv8up')

In [31]:
sessionid, date, time = generate_session_details()

In [32]:
sessionid

'session_20250802094535733990-iv1f76'

In [33]:
date

'2025-08-02'

In [34]:
time

'09:45:35'